In [ ]:
!pip install datasets
!pip install groq
!pip install google-colab
import os
from groq import Groq
from datasets import load_dataset
import json
from google.colab import userdata

# Initialize Groq client with the API key
client = Groq(api_key=userdata.get('GROQ_API_KEY'))




In [ ]:
# Function to generate translations using the Groq API

#Works but translation kind of sucks
def generate_translation(input_text, target_language):
    prompt = f"TRANSLATE TO {target_language}. DO NOT PROVIDE ANY COMMENTARY OR NOTES. JUST PROVIDE THE DIRECT TRANSLATION TO {target_language}: '{input_text}'"
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"You are a professional translator who specializes in translating from English to {target_language}",
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="mixtral-8x7b-32768",
    )
    translation = chat_completion.choices[0].message.content.strip()
    return translation

# Function to process each item in the dataset and translate it
def process_item(item, target_languages):
    translations = []
    try:
        original_text = [{'system':item[0]['value']},{'human':item[1]['value']},{'gpt':item[2]['value']}]
    except IndexError:
        print(f"Error processing item: {item}")
        return translations
    translation_for_each_lang = []
    for lang in target_languages:
        system_translation = generate_translation(item[0]['value'], lang) if len(item) > 0 else ''
        human_translation = generate_translation(item[1]['value'], lang) if len(item) > 1 else ''
        gpt_translation = generate_translation(item[2]['value'], lang) if len(item) > 2 else ''
        translation_for_each_lang.append({
            'language': lang,
            'system_translation': system_translation,
            'human_translation': human_translation,
            'gpt_translation': gpt_translation
        })
    translations.append({'original': original_text, 'translations': translation_for_each_lang})
    print(translations)
    return translations
def main():
    # Load the MoD dataset
    dataset = load_dataset("Crystalcareai/MoD")['train'][:25]

    # Define target languages for translation
    target_languages = ['Indonesian', 'Thai', 'Vietnamese']
    results = [process_item(item, target_languages) for item in dataset['conversations']]

    # Combine all translations
    all_translations = [result for sublist in results for result in sublist]

    # Save translations to a JSON file
    with open('mod_translations.json', 'w', encoding='utf-8') as json_file:
        json.dump(all_translations, json_file, ensure_ascii=False, indent=4)

    print("Translation process completed and saved to mod_translations.json.")

if __name__ == "__main__":
    main()

In [7]:

# Function to generate translations using the Groq API
#Translate Alpaca 1k
def generate_translation(input_text, target_language):
    prompt = f"TRANSLATE TO {target_language}. DO NOT PROVIDE ANY COMMENTARY OR NOTES. JUST PROVIDE THE DIRECT TRANSLATION TO {target_language}: '{input_text}'"
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"You are a professional translator",
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="mixtral-8x7b-32768",
    )
    translation = chat_completion.choices[0].message.content.strip()
    return translation

# Function to process each item in the dataset and translate it
def process_item(item, target_languages):
    translations = []
    original_text = {
        'input': item['input'] if item['input'] else '',
        'output': item['output'] if item['output'] else '',
        'instruction': item['instruction'] if item['instruction'] else ''
    }
    translation_for_each_lang = []
    for lang in target_languages:
        input_translation = generate_translation(original_text['input'], lang) if len(original_text['input']) > 0 else ''
        output_translation = generate_translation(original_text['output'], lang) if len(original_text['output']) > 0 else ''
        instruction_translation = generate_translation(original_text['instruction'], lang) if len(original_text['instruction']) > 0 else ''
        translation_for_each_lang.append({
            'language': lang,
            'input_translation': input_translation,
            'output_translation': output_translation,
            'instruction_translation': instruction_translation
        })
    translations.append({'original': original_text, 'translations': translation_for_each_lang})
    print(translations)
    return translations
def create_conversations(data):
    conversations = []
    for i in range(len(data['input'])):
        conversations.append({
            'input': data['input'][i],
            'output': data['output'][i],
            'instruction': data['instruction'][i]
        })
    return conversations
def main():
    # Load the MoD dataset
    dataset = load_dataset("Crystalcareai/alpaca-cleaned-1k")['train'][:25]
    # Define target languages for translation
    target_languages = ['Indonesian', 'Thai', 'Vietnamese']
    conversations = create_conversations(dataset)
    results = [process_item(conversation, target_languages) for conversation in conversations]

    # Combine all translations
    all_translations = [result for sublist in results for result in sublist]

    # Save translations to a JSON file
    with open('mod_translations.json', 'w', encoding='utf-8') as json_file:
        json.dump(all_translations, json_file, ensure_ascii=False, indent=4)

    print("Translation process completed and saved to mod_translations.json.")

if __name__ == "__main__":
    main()

[{'original': {'input': 'Using Google Maps can be really helpful.', 'output': 'Using Google Maps can provide numerous benefits such as accurate navigation, real-time traffic updates, and easy exploration of new destinations.', 'instruction': 'Change the given sentence into a statement about the benefits of something.'}, 'translations': [{'language': 'Indonesian', 'input_translation': 'Menggunakan Google Maps dapat sangat bermanfaat.\nUser manual translation services.', 'output_translation': 'Menggunakan Google Maps dapat menyediakan banyak manfaat seperti navigasi yang tepat, pembaruan lalulintas real-time, dan eksplorasi mudah destinasi baru.\n\n(Note: Indonesian language does not have a direct translation for the word "benefits". Thus, I used the Indonesian word "manfaat" which has a similar meaning but slightly more general, indicating "useful things/advantages/utilities/profits/etc.")', 'instruction_translation': "Mengubah kalimat tersebut menjadi pernyataan mengenai keuntungan ses

[{'from': 'system', 'value': 'You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.'}, {'from': 'human', 'value': 'Summarize this article:\n\nThe company\'s flotation is the largest on the London market so far this year. Its shares were initially offered at 240p each, after its private equity owners sold a 51% stake in it.\nWorldpay processes more than 30 million mobile, online and in-store transactions every day.\nThe company employs about 4,500 staff, mostly in the UK and the US.\nCompany chief executive Philip Jansen described Worldpay\'s listing as "a significant milestone".\nHe added that more than £1bn had been invested in the company, "helping [it] become an advanced technology-led organisation".\nWorldpay was sold by the Royal Bank of Scotland in 2010 to its current owners, Advent International and Bain Capital. In terms of money raised, the flotation also means it is Europe\'s largest private equity-backed listing.\nIn a 

In [ ]:



def generate_translation(batch_texts, target_language):
    # Batch tokenization and move tensors to the same device as the model
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True)#.to(device)
    with torch.no_grad():
        translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[target_language])
    translated_texts = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    return translated_texts

def process_conversations(batch):
    target_languages = ["tgl_Latn"]
    translations = []

    for item in batch:
        for system_text in item["system"]:
            for instruction_text in item["instruction"]:
                for output_text in item["output"]:
                    translations.append({
                        "system": system_text,
                        "instruction": instruction_text,
                        "output": output_text,
                        "instruction_translations": generate_translation([instruction_text], target_languages[0]),
                        "output_translations": generate_translation([output_text], target_languages[0]),
                    })

    return translations

def main():
    dataset = load_dataset("Crystalcareai/slimorca-dedup-alpaca-100k", split='train')

    # Use dataset.map with batched processing for efficiency
    result = dataset.map(process_conversations, batched=True, batch_size=16)

    # Save results
    result.to_json('translated_conversations.json')

    print("Filtered translation process completed. Check 'translated_conversations.json'.")

if __name__ == "__main__":
    main()
